# Particle Swarm Optimization (PSO)

MGSurvE also contains a beta implementation of a PSO algorithm as an alternative to the base GA.

Special thanks to Lillian Weng, Ayden Salazar, Xingli Yu, Topiltzin Hernandes, Joanna Yoo, and Elijah Bartolome for this implementation.

In [18]:
# Fix PROJ path ---------------------------------------------------------------
import os; 
os.environ['PROJ_LIB']=r'/opt/conda/pkgs/proj-9.2.1-ha5fc9e9_0/share/proj'
# Load libraries --------------------------------------------------------------
from os import path
import numpy as np
import pandas as pd
import MGSurvE as srv
from math import sqrt, inf
import matplotlib.pyplot as plt
# Setup data path and ID ------------------------------------------------------
(ID, LND_TOP, OUT_PTH) = ('pso', 'Grid', './out/')
srv.makeFolder(OUT_PTH)

This time we will allow the setup of three different types of synthetic landscapes:
* `"Grid"`: A regular grid of points laid across a rectangular setup.
* `"Uniform"`: A sample of points uniformly sampled from a rectangular set of coordinates.
* `"Ring"`: A random sample of points laid along a ring with an iner and outer radius.

In [19]:
PTS_NUM = 400
RADII = (425, 500)
BBOX = ((-500, 500), (-350, 350))
# Generate Synthetic Pointset -------------------------------------------------
if LND_TOP == 'Grid':
    (ptsNum, ptsTypes) = (int(sqrt(PTS_NUM)), 1)
    xy = srv.ptsRegularGrid(PTS_NUM, (BBOX[0], BBOX[0])).T
elif LND_TOP == 'Uniform':
    (ptsNum, ptsTypes) = (PTS_NUM, 1)
    xy = srv.ptsRandUniform(ptsNum, BBOX).T
elif LND_TOP == 'Ring':
    (ptsNum, radii, ptsTypes) = (PTS_NUM, RADII, 1)
    xy = srv.ptsDonut(ptsNum, radii).T
pType = np.random.choice(ptsTypes, xy.shape[1])
points = pd.DataFrame({'x': xy[0], 'y': xy[1], 't': pType})
mKer = {'params': [.075, 1.0e-10, inf], 'zeroInflation': .75}

In [20]:
# Traps -----------------------------------------------------------------------
traps = pd.DataFrame({
    'x': [0, 0, 0, 0, 0], 
    'y': [0, 0, 0, 0, 0],
    't': [0, 1, 0, 1, 0], 
    'f': [0, 0, 0, 0, 0]
})
tKer = {
    0: {'kernel': srv.exponentialDecay, 'params': {'A': .75, 'b': .025}},
    1: {'kernel': srv.exponentialDecay, 'params': {'A': .50, 'b': .015}}
}
# Landscape -------------------------------------------------------------------
lnd = srv.Landscape(
    points, 
    kernelParams=mKer,
    traps=traps, trapsKernels=tKer
)
bbox = lnd.getBoundingBox()

In [ ]:
GENS = 100
# PSO -------------------------------------------------------------------------
(GENS, PARTS, SPD, PHI) = (
    GENS, traps.shape[0]*20,
    (-max(max(bbox))/30, max(max(bbox))/30),
    (max(max(bbox))/15, max(max(bbox))/15)
)
pso = srv.Particle_Swarm(
    lnd=lnd, traps=traps,
    num_particles=PARTS, num_gens=GENS, 
    p_min=min(bbox[0][0], bbox[1][0]), p_max=max(bbox[1][0], bbox[1][1]),  
    s_min=SPD[0], s_max=SPD[1],
    phi1=PHI[0], phi2=PHI[1],
    optimFunctionArgs={'inner': np.sum, 'outer': np.max}
)
(pop, logbook, _) = pso.evaluate()
best = list(logbook[logbook['min']==min(logbook['min'])]['traps'])[0]
bestTraps = np.reshape(best, (-1, 2))
lnd.updateTrapsCoords(bestTraps)


/Users/sanchez.hmsc/miniconda3/envs/MGSurvE/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/sanchez.hmsc/miniconda3/envs/MGSurvE/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Particle' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	evals	min    	avg    	max    	std    	best	traps                                                                                                                                                                                                  
0  	100  	179.317	455.134	962.807	176.104	99  	[304.9155587241214, 472.42582507134614, -313.8689897018541, 458.9395137763562, 416.5949342945118, -288.0619783932892, -259.14830564531735, -130.54864907985117, -23.272617780709282, 38.98085030319302]
1  	100  	183.18 	448.168	992.257	171.91 	99  	[321.5822253907881, 455.75915840467945, -297.2023230351874, 475.6061804430229, 399.9282676278451, -271.39531172662254, -275.81497231198404, -147.21531574651783, -39.939284447375954, 55.647516969859694]
2  	100  	168.542	440.147	1014.84	168.776	99  	[338.2488920574548, 439.09249173801277, -280.53565636852073, 492.27284710968956, 383.26160096117843, -254.72864505995588, -292.4816389786507, -163.8819824131845, -56.605951114042625, 72.31418363652637]
3  	100 

In [ ]:
# Export Results --------------------------------------------------------------
dta = pd.DataFrame(logbook)
srv.dumpLandscape(lnd, OUT_PTH, '{}_{}-TRP'.format(ID, LND_TOP), fExt='pkl')
srv.exportLog(logbook, OUT_PTH, '{}_{}-LOG'.format(ID, LND_TOP))
# Plot Results ----------------------------------------------------------------
(fig, ax) = plt.subplots(1, 1, figsize=(15, 15), sharey=False)
lnd.plotSites(fig, ax, size=100)
# lnd.plotMigrationNetwork(fig, ax, alphaMin=.6, lineWidth=25)
lnd.plotTraps(fig, ax)
srv.plotFitness(fig, ax, min(logbook['min']), zorder=30)
srv.plotClean(fig, ax, bbox=lnd.landLimits)
fig.savefig(
    path.join(OUT_PTH, '{}_{}-TRP.png'.format(ID, LND_TOP)),
    facecolor='w', bbox_inches='tight', 
    pad_inches=0, dpi=300
)